In [ ]:
import requests
import os
import time
from datetime import date

# Threads API 설정
ACCESS_TOKEN = "YOUR_ACCESS_TOKEN"  # Threads API 인증 액세스 토큰
USER_ID = "YOUR_USER_ID"  # Threads 사용자 ID
BASE_URL = "https://graph.threads.net/v1.0"

# 오늘 날짜와 파일 경로 설정
TODAY = date.today().strftime("%Y-%m-%d")
selected_domain = "esg"  # 도메인 이름 설정
file_path = os.path.join(".", "kakao", f"{TODAY}_kakao_letter_{selected_domain}.txt")

# TXT 파일 읽기
try:
    with open(file_path, "r", encoding="utf-8") as file:
        post_text = file.read().strip()  # TXT 파일에서 내용을 읽어옵니다
except FileNotFoundError:
    print(f"파일을 찾을 수 없습니다: {file_path}")
    exit(1)
except Exception as e:
    print(f"파일을 읽는 데 실패했습니다: {e}")
    exit(1)

# Threads API 함수
def create_media_container(access_token, user_id, text):
    """
    Threads 미디어 컨테이너 생성
    """
    url = f"{BASE_URL}/{user_id}/threads"
    payload = {
        "text": text,
        "media_type": "TEXT",  # 텍스트 게시물
    }
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }

    response = requests.post(url, json=payload, headers=headers)

    if response.status_code == 200:
        container_id = response.json().get("id")
        print(f"미디어 컨테이너 생성 성공: {container_id}")
        return container_id
    else:
        print(f"미디어 컨테이너 생성 실패: {response.status_code}, {response.text}")
        return None

def publish_media_container(access_token, user_id, container_id):
    """
    Threads 미디어 컨테이너 게시
    """
    url = f"{BASE_URL}/{user_id}/threads_publish"
    payload = {"creation_id": container_id}
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }

    response = requests.post(url, json=payload, headers=headers)

    if response.status_code == 200:
        post_id = response.json().get("id")
        print(f"게시물 작성 성공: {post_id}")
        return post_id
    else:
        print(f"게시물 작성 실패: {response.status_code}, {response.text}")
        return None

# 1단계: 미디어 컨테이너 생성
container_id = create_media_container(ACCESS_TOKEN, USER_ID, post_text)

if container_id:
    # 컨테이너 생성 후 5초 대기
    time.sleep(5)
    
    # 2단계: 미디어 컨테이너 게시
    publish_media_container(ACCESS_TOKEN, USER_ID, container_id)
